In [19]:
import pythainlp
from pythainlp import sent_tokenize, word_tokenize
from pythainlp import word_tokenize, Tokenizer
from pythainlp.util.trie import Trie, dict_trie
from pythainlp.corpus.common import thai_words
from pythainlp.util import normalize
from pythainlp.tokenize import word_tokenize
import pandas as pd
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [20]:
df_stock = pd.read_excel('input/ai_mapping_stock.xlsx',sheet_name='Stock')[["OtherDescription","SKUCode"]]
df_stock.head()

,OtherDescription,SKUCode
0,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...,1-GDS-HYGIE-000000027
1,ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...,1-GDS-HYGIE-000000090
2,ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...,1-GDS-HYGIE-000000042
3,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...,1-GDS-HYGIE-000000034
4,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เซ้นท์ น้ำยาปรับผ้...,1-GDS-HYGIE-000000058


In [21]:
df_item = pd.read_excel('input/ai_mapping_stock.xlsx',sheet_name='Item')[["SKUCode","Quantity"]]
df_item.head()

,SKUCode,Quantity
0,1-GDS-HYGIE-000000031,464
1,1-GDS-HYGIE-000000120,432
2,1-GDS-HYGIE-000000119,228
3,1-GDS-HYGIE-000000152,1200
4,1-GDS-HYGIE-000000040,1480


In [22]:
df_vendor = pd.read_excel('input/vendor_data.xlsx')
df_vendor.head()

,ProductCode,ProductName,Qty,Pack,Weight
0,1101155228,วิกซอล โกลด์ 900 (2+1),12,1,12.5
1,1102043121,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,12,1,9.8
2,1201033318,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,12,1,8.5
3,1201053312,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,12,1,8.5
4,1201133155,วิช-ถูพื้น เนเจอร์ ชมพู 5200,4,1,21.9


In [23]:
df_bzb = pd.merge(df_stock, df_item, how='left', on='SKUCode')
df_bzb.head()

,OtherDescription,SKUCode,Quantity
0,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...,1-GDS-HYGIE-000000027,12
1,ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...,1-GDS-HYGIE-000000090,12
2,ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...,1-GDS-HYGIE-000000042,12
3,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...,1-GDS-HYGIE-000000034,12
4,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เซ้นท์ น้ำยาปรับผ้...,1-GDS-HYGIE-000000058,12


In [24]:
stopwords = ['(', ')', '-', '_', '', ' ']

In [25]:
with open("quantity.json", encoding='utf-8') as js:
    quantity_word_list = json.load(js)

In [26]:
quantity_words = list(quantity_word_list.keys())

trie = dict_trie(dict_source=quantity_words)

custom_tokenizer = Tokenizer(custom_dict=trie, engine='newmm')

In [27]:
def quantity_cal (word_quantity):
    qty = 0
    for i in word_quantity:
        if i in quantity_words:
            qty = quantity_word_list[i]
        else:
            qty = 1
    return qty

## Tokenize Vendor Table

In [29]:
# Normalize text
df_vendor['Vendor_Name_Clean01'] = df_vendor.apply(lambda row: normalize(row['ProductName']), axis=1)
 
# Word segmentation
df_vendor['Vendor_Name_Clean02'] = df_vendor.apply(lambda row: custom_tokenizer.word_tokenize(row['Vendor_Name_Clean01']), axis=1)
 
# Remove stopwords
df_vendor['Vendor_Name_Clean03'] = df_vendor.apply(lambda row: " ".join([i for i in row['Vendor_Name_Clean02'] if i not in stopwords]), axis=1)

# Calculate Quantity to match BZB table
df_vendor['BZB_Qty'] = df_vendor['Qty']/df_vendor['Vendor_Name_Clean02'].apply(quantity_cal)
df_vendor['BZB_Qty'] = df_vendor['BZB_Qty'].astype(int)

In [30]:
df_vendor.head()

,ProductCode,ProductName,Qty,Pack,Weight,Vendor_Name_Clean01,Vendor_Name_Clean02,Vendor_Name_Clean03,BZB_Qty
0,1101155228,วิกซอล โกลด์ 900 (2+1),12,1,12.5,วิกซอล โกลด์ 900 (2+1),"[วิกซอล, , โกลด์, , 900, , (2+1)]",วิกซอล โกลด์ 900 (2+1),4
1,1102043121,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,12,1,9.8,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,"[วิกซอล, , ออกซี่, , มิ้นท์, , 700, , มล, ...",วิกซอล ออกซี่ มิ้นท์ 700 มล . 3S,4
2,1201033318,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,12,1,8.5,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,"[วิช, -, ถูพื้น, , ทรีดี, , ชมพู, , 650, ,...",วิช ถูพื้น ทรีดี ชมพู 650 ซอง,12
3,1201053312,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,12,1,8.5,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,"[วิช, -, ถูพื้น, , ทรีดี, , ฟ้า, , 650, , ...",วิช ถูพื้น ทรีดี ฟ้า 650 ซอง,12
4,1201133155,วิช-ถูพื้น เนเจอร์ ชมพู 5200,4,1,21.9,วิช-ถูพื้น เนเจอร์ ชมพู 5200,"[วิช, -, ถูพื้น, , เนเจอร์, , ชมพู, , 5200]",วิช ถูพื้น เนเจอร์ ชมพู 5200,4


In [31]:
df_vendor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ProductCode          50 non-null     int64  
 1   ProductName          50 non-null     object 
 2   Qty                  50 non-null     int64  
 3   Pack                 50 non-null     int64  
 4   Weight               50 non-null     float64
 5   Vendor_Name_Clean01  50 non-null     object 
 6   Vendor_Name_Clean02  50 non-null     object 
 7   Vendor_Name_Clean03  50 non-null     object 
 8   BZB_Qty              50 non-null     int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 3.6+ KB


## Tokenize BZB Table

In [32]:
# Normalize text
df_bzb['BZB_Name_Clean01'] = df_bzb.apply(lambda row: normalize(row['OtherDescription']), axis=1)
 
# Word segmentation
df_bzb['BZB_Name_Clean02'] = df_bzb.apply(lambda row: custom_tokenizer.word_tokenize(row['BZB_Name_Clean01']), axis=1)
 
# Remove stopwords
df_bzb['BZB_Name_Clean03'] = df_bzb.apply(lambda row: " ".join([i for i in row['BZB_Name_Clean02'] if i not in stopwords]), axis=1)

In [33]:
df_bzb.head()

,OtherDescription,SKUCode,Quantity,BZB_Name_Clean01,BZB_Name_Clean02,BZB_Name_Clean03
0,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...,1-GDS-HYGIE-000000027,12,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...,"[ไฮยีน, , เอ็กซ์เพิร์ท, , แคร์, , ไลฟ์เนเจอ...",ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...
1,ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...,1-GDS-HYGIE-000000090,12,ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...,"[ไฮยีน, , เอ็กซ์เพิร์ท, , แคร์, , น้ำยาปรับ...",ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...
2,ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...,1-GDS-HYGIE-000000042,12,ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...,"[ไฮยีน, , เอ็กซ์เพิร์ท, , แคร์, , น้ำยาปรับ...",ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...
3,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...,1-GDS-HYGIE-000000034,12,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...,"[ไฮยีน, , เอ็กซ์เพิร์ท, , แคร์, , ไลฟ์เนเจอ...",ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...
4,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เซ้นท์ น้ำยาปรับผ้...,1-GDS-HYGIE-000000058,12,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เซ้นท์ น้ำยาปรับผ้...,"[ไฮยีน, , เอ็กซ์เพิร์ท, , แคร์, , ไลฟ์เซ้นท...",ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เซ้นท์ น้ำยาปรับผ้...


In [34]:
def concatenate_matching_indices(dfA, dfB, colA, colB):
    """
    Compare specific columns in two DataFrames and return tableA with an additional column
    that contains the indices from tableB that match the value of each row in tableA.
    
    Parameters:
    - tableA: The first DataFrame
    - colA: The name of the column in the first DataFrame to compare
    - tableB: The second DataFrame
    - colB: The name of the column in the second DataFrame to compare
    
    Returns:
    - A DataFrame with an additional column 'matching_indices' containing lists of matching indices in tableB
    """
    if colA not in dfA.columns:
        raise ValueError(f"Column '{colA}' not found in tableA")
    if colB not in dfB.columns:
        raise ValueError(f"Column '{colB}' not found in tableB")
    
    matching_indices = []
    
    for valueA in dfA[colA]:
        matching_indices.append(dfB.index[dfB[colB] == valueA].tolist())
    
    dfA['matching_indices'] = matching_indices
    return dfA

In [35]:
concatenate_matching_indices(df_vendor, df_bzb, 'BZB_Qty', 'Quantity')
df_vendor.head()

,ProductCode,ProductName,Qty,Pack,Weight,Vendor_Name_Clean01,Vendor_Name_Clean02,Vendor_Name_Clean03,BZB_Qty,matching_indices
0,1101155228,วิกซอล โกลด์ 900 (2+1),12,1,12.5,วิกซอล โกลด์ 900 (2+1),"[วิกซอล, , โกลด์, , 900, , (2+1)]",วิกซอล โกลด์ 900 (2+1),4,"[27, 28, 29, 30, 31, 32, 37, 48]"
1,1102043121,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,12,1,9.8,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,"[วิกซอล, , ออกซี่, , มิ้นท์, , 700, , มล, ...",วิกซอล ออกซี่ มิ้นท์ 700 มล . 3S,4,"[27, 28, 29, 30, 31, 32, 37, 48]"
2,1201033318,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,12,1,8.5,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,"[วิช, -, ถูพื้น, , ทรีดี, , ชมพู, , 650, ,...",วิช ถูพื้น ทรีดี ชมพู 650 ซอง,12,"[0, 1, 2, 3, 4, 5, 7, 8, 9, 16, 17, 19, 20, 21..."
3,1201053312,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,12,1,8.5,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,"[วิช, -, ถูพื้น, , ทรีดี, , ฟ้า, , 650, , ...",วิช ถูพื้น ทรีดี ฟ้า 650 ซอง,12,"[0, 1, 2, 3, 4, 5, 7, 8, 9, 16, 17, 19, 20, 21..."
4,1201133155,วิช-ถูพื้น เนเจอร์ ชมพู 5200,4,1,21.9,วิช-ถูพื้น เนเจอร์ ชมพู 5200,"[วิช, -, ถูพื้น, , เนเจอร์, , ชมพู, , 5200]",วิช ถูพื้น เนเจอร์ ชมพู 5200,4,"[27, 28, 29, 30, 31, 32, 37, 48]"


In [36]:
df_vendor

,ProductCode,ProductName,Qty,Pack,Weight,Vendor_Name_Clean01,Vendor_Name_Clean02,Vendor_Name_Clean03,BZB_Qty,matching_indices
0,1101155228,วิกซอล โกลด์ 900 (2+1),12,1,12.50,วิกซอล โกลด์ 900 (2+1),"[วิกซอล, , โกลด์, , 900, , (2+1)]",วิกซอล โกลด์ 900 (2+1),4,"[27, 28, 29, 30, 31, 32, 37, 48]"
1,1102043121,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,12,1,9.80,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,"[วิกซอล, , ออกซี่, , มิ้นท์, , 700, , มล, ...",วิกซอล ออกซี่ มิ้นท์ 700 มล . 3S,4,"[27, 28, 29, 30, 31, 32, 37, 48]"
2,1201033318,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,12,1,8.50,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,"[วิช, -, ถูพื้น, , ทรีดี, , ชมพู, , 650, ,...",วิช ถูพื้น ทรีดี ชมพู 650 ซอง,12,"[0, 1, 2, 3, 4, 5, 7, 8, 9, 16, 17, 19, 20, 21..."
3,1201053312,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,12,1,8.50,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,"[วิช, -, ถูพื้น, , ทรีดี, , ฟ้า, , 650, , ...",วิช ถูพื้น ทรีดี ฟ้า 650 ซอง,12,"[0, 1, 2, 3, 4, 5, 7, 8, 9, 16, 17, 19, 20, 21..."
4,1201133155,วิช-ถูพื้น เนเจอร์ ชมพู 5200,4,1,21.90,วิช-ถูพื้น เนเจอร์ ชมพู 5200,"[วิช, -, ถูพื้น, , เนเจอร์, , ชมพู, , 5200]",วิช ถูพื้น เนเจอร์ ชมพู 5200,4,"[27, 28, 29, 30, 31, 32, 37, 48]"
5,1301033361,ไฮยีน-บลีช ชมพู 1500,6,1,11.20,ไฮยีน-บลีช ชมพู 1500,"[ไฮยีน, -, บลีช, , ชมพู, , 1500]",ไฮยีน บลีช ชมพู 1500,6,"[23, 35, 38, 39, 40, 41]"
6,1302043299,ไฮยีน-บลีชผ้าสี 1000,12,1,13.90,ไฮยีน-บลีชผ้าสี 1000,"[ไฮยีน, -, บลีชผ้าสี, , 1000]",ไฮยีน บลีชผ้าสี 1000,12,"[0, 1, 2, 3, 4, 5, 7, 8, 9, 16, 17, 19, 20, 21..."
7,1303053559,ไฮยีน ปากกาขจัดคราบ ฟ้า 10,24,2,0.60,ไฮยีน ปากกาขจัดคราบ ฟ้า 10,"[ไฮยีน, , ปากกาขจัดคราบ, , ฟ้า, , 10]",ไฮยีน ปากกาขจัดคราบ ฟ้า 10,24,"[47, 49]"
8,1307023536,ไฮยีน ซักผ้า ฟอร์เอเวอร์ บลูม 1400,8,1,12.40,ไฮยีน ซักผ้า ฟอร์เอเวอร์ บลูม 1400,"[ไฮยีน, , ซักผ้า, , ฟอร์เอเวอร์, , บลูม, ,...",ไฮยีน ซักผ้า ฟอร์เอเวอร์ บลูม 1400,8,"[10, 24, 45]"
9,1307033724,ไฮยีนซักผ้า เลิฟลี่ 2800,8,2,25.40,ไฮยีนซักผ้า เลิฟลี่ 2800,"[ไฮยีนซักผ้า, , เลิฟลี่, , 2800]",ไฮยีนซักผ้า เลิฟลี่ 2800,8,"[10, 24, 45]"


In [37]:
def calculate_max_cosine_similarity(tableA, colA, tableB, colB, match_col, return_col):
    """
    Calculate the cosine similarity between text in tableA and tableB based on matching indices,
    and concatenate the maximum cosine similarity and corresponding value from tableB to tableA.
    
    Parameters:
    - tableA: The first DataFrame
    - colA: The name of the text column in the first DataFrame
    - tableB: The second DataFrame
    - colB: The name of the text column in the second DataFrame
    - match_col: The name of the column in tableA containing the matching indices
    - return_col: The name of the column in tableB from which to return the value corresponding to the max similarity
    
    Returns:
    - A DataFrame with additional columns 'max_cosine_similarity' and the specified return column value
    """
    # Ensure the matching column exists in tableA
    if match_col not in tableA.columns:
        raise ValueError(f"Column '{match_col}' not found in tableA")
    
    # Ensure the return column exists in tableB
    if return_col not in tableB.columns:
        raise ValueError(f"Column '{return_col}' not found in tableB")

    max_similarities = []
    return_values = []
    vectorizer = TfidfVectorizer()

    for index, row in tableA.iterrows():
        matching_indices = row[match_col]
        textA = row[colA]
        
        if not matching_indices:  # If matching_indices is an empty list
            textsB = tableB[colB].tolist()
            return_values_full = tableB[return_col].tolist()
        else:
            textsB = tableB.loc[matching_indices, colB].tolist()
            return_values_full = tableB.loc[matching_indices, return_col].tolist()

        # Combine the text from tableA and matching texts from tableB for vectorization
        all_texts = [textA] + textsB
        tfidf_matrix = vectorizer.fit_transform(all_texts)

        # Calculate cosine similarities with the first row being the text from tableA
        cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])
        
        # Get the maximum cosine similarity and the corresponding value from tableB
        max_similarity_index = cosine_sim.argmax()
        max_similarity = cosine_sim.max()
        max_return_value = return_values_full[max_similarity_index]
        
        max_similarities.append(max_similarity)
        return_values.append(max_return_value)
    
    tableA['max_cosine_similarity'] = max_similarities
    tableA[f'return_{return_col}'] = return_values
    return tableA

In [38]:
calculate_max_cosine_similarity(df_vendor, 'Vendor_Name_Clean03', df_bzb, 'BZB_Name_Clean03', 'matching_indices', 'SKUCode')
df_vendor.head()

,ProductCode,ProductName,Qty,Pack,Weight,Vendor_Name_Clean01,Vendor_Name_Clean02,Vendor_Name_Clean03,BZB_Qty,matching_indices,max_cosine_similarity,return_SKUCode
0,1101155228,วิกซอล โกลด์ 900 (2+1),12,1,12.5,วิกซอล โกลด์ 900 (2+1),"[วิกซอล, , โกลด์, , 900, , (2+1)]",วิกซอล โกลด์ 900 (2+1),4,"[27, 28, 29, 30, 31, 32, 37, 48]",0.482393,1-GDS-VIXOL-000000014
1,1102043121,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,12,1,9.8,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,"[วิกซอล, , ออกซี่, , มิ้นท์, , 700, , มล, ...",วิกซอล ออกซี่ มิ้นท์ 700 มล . 3S,4,"[27, 28, 29, 30, 31, 32, 37, 48]",0.296545,1-GDS-VIXOL-000000025
2,1201033318,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,12,1,8.5,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,"[วิช, -, ถูพื้น, , ทรีดี, , ชมพู, , 650, ,...",วิช ถูพื้น ทรีดี ชมพู 650 ซอง,12,"[0, 1, 2, 3, 4, 5, 7, 8, 9, 16, 17, 19, 20, 21...",0.198344,1-GDS-WHIZ0-000000004
3,1201053312,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,12,1,8.5,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,"[วิช, -, ถูพื้น, , ทรีดี, , ฟ้า, , 650, , ...",วิช ถูพื้น ทรีดี ฟ้า 650 ซอง,12,"[0, 1, 2, 3, 4, 5, 7, 8, 9, 16, 17, 19, 20, 21...",0.135273,1-GDS-WHIZ0-000000005
4,1201133155,วิช-ถูพื้น เนเจอร์ ชมพู 5200,4,1,21.9,วิช-ถูพื้น เนเจอร์ ชมพู 5200,"[วิช, -, ถูพื้น, , เนเจอร์, , ชมพู, , 5200]",วิช ถูพื้น เนเจอร์ ชมพู 5200,4,"[27, 28, 29, 30, 31, 32, 37, 48]",0.139477,1-GDS-HYGIE-000000110


In [39]:
def recalculate_similarity_if_below_threshold(tableA, colA, tableB, colB, match_col, threshold, return_col):
    """
    Recalculate the cosine similarity for rows in tableA where max_cosine_similarity is below the threshold.
    Compare the text in tableA with all text in tableB instead of just matching indices.
    Update tableA with the maximum cosine similarity and a new value from tableB.

    Parameters:
    - tableA: The first DataFrame
    - colA: The name of the text column in the first DataFrame
    - tableB: The second DataFrame
    - colB: The name of the text column in the second DataFrame
    - match_col: The name of the column in tableA containing the matching indices
    - threshold: The threshold value below which to recalculate the cosine similarity
    - return_col: The name of the column in tableB from which to return the value corresponding to the max similarity
    
    Returns:
    - A DataFrame with updated 'max_cosine_similarity' values and the new return column value
    """
    # Ensure the matching column exists in tableA
    if match_col not in tableA.columns:
        raise ValueError(f"Column '{match_col}' not found in tableA")
    
    # Ensure the columns exist in both DataFrames
    if colA not in tableA.columns:
        raise ValueError(f"Column '{colA}' not found in tableA")
    if colB not in tableB.columns:
        raise ValueError(f"Column '{colB}' not found in tableB")
    if return_col not in tableB.columns:
        raise ValueError(f"Column '{return_col}' not found in tableB")

    # Get the rows where the current max_cosine_similarity is below the threshold
    rows_to_recalculate = tableA[tableA['max_cosine_similarity'] < threshold].index
    
    vectorizer = TfidfVectorizer()
    
    for index in rows_to_recalculate:
        row = tableA.loc[index]
        textA = row[colA]
        
        # Recalculate with all texts from tableB
        textsB = tableB[colB].tolist()
        return_values_full = tableB[return_col].tolist()
        tfidf_matrix = vectorizer.fit_transform([textA] + textsB)

        # Calculate cosine similarities with the first row being the text from tableA
        cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])
        
        # Get the new maximum cosine similarity and the corresponding value from tableB
        max_similarity_index = cosine_sim.argmax()
        new_max_similarity = cosine_sim.max()
        max_return_value = return_values_full[max_similarity_index]
        
        # Update the tableA DataFrame
        tableA.at[index, 'max_cosine_similarity'] = new_max_similarity
        tableA.at[index, f'return_{return_col}'] = max_return_value
    
    return tableA

In [40]:
recalculate_similarity_if_below_threshold(df_vendor, 'Vendor_Name_Clean03', df_bzb, 'BZB_Name_Clean03', 'matching_indices', 0.1, 'SKUCode')
df_vendor.head()

,ProductCode,ProductName,Qty,Pack,Weight,Vendor_Name_Clean01,Vendor_Name_Clean02,Vendor_Name_Clean03,BZB_Qty,matching_indices,max_cosine_similarity,return_SKUCode
0,1101155228,วิกซอล โกลด์ 900 (2+1),12,1,12.5,วิกซอล โกลด์ 900 (2+1),"[วิกซอล, , โกลด์, , 900, , (2+1)]",วิกซอล โกลด์ 900 (2+1),4,"[27, 28, 29, 30, 31, 32, 37, 48]",0.482393,1-GDS-VIXOL-000000014
1,1102043121,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,12,1,9.8,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,"[วิกซอล, , ออกซี่, , มิ้นท์, , 700, , มล, ...",วิกซอล ออกซี่ มิ้นท์ 700 มล . 3S,4,"[27, 28, 29, 30, 31, 32, 37, 48]",0.296545,1-GDS-VIXOL-000000025
2,1201033318,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,12,1,8.5,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,"[วิช, -, ถูพื้น, , ทรีดี, , ชมพู, , 650, ,...",วิช ถูพื้น ทรีดี ชมพู 650 ซอง,12,"[0, 1, 2, 3, 4, 5, 7, 8, 9, 16, 17, 19, 20, 21...",0.198344,1-GDS-WHIZ0-000000004
3,1201053312,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,12,1,8.5,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,"[วิช, -, ถูพื้น, , ทรีดี, , ฟ้า, , 650, , ...",วิช ถูพื้น ทรีดี ฟ้า 650 ซอง,12,"[0, 1, 2, 3, 4, 5, 7, 8, 9, 16, 17, 19, 20, 21...",0.135273,1-GDS-WHIZ0-000000005
4,1201133155,วิช-ถูพื้น เนเจอร์ ชมพู 5200,4,1,21.9,วิช-ถูพื้น เนเจอร์ ชมพู 5200,"[วิช, -, ถูพื้น, , เนเจอร์, , ชมพู, , 5200]",วิช ถูพื้น เนเจอร์ ชมพู 5200,4,"[27, 28, 29, 30, 31, 32, 37, 48]",0.139477,1-GDS-HYGIE-000000110


In [41]:
df_bzb.head()

,OtherDescription,SKUCode,Quantity,BZB_Name_Clean01,BZB_Name_Clean02,BZB_Name_Clean03
0,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...,1-GDS-HYGIE-000000027,12,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...,"[ไฮยีน, , เอ็กซ์เพิร์ท, , แคร์, , ไลฟ์เนเจอ...",ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...
1,ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...,1-GDS-HYGIE-000000090,12,ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...,"[ไฮยีน, , เอ็กซ์เพิร์ท, , แคร์, , น้ำยาปรับ...",ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...
2,ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...,1-GDS-HYGIE-000000042,12,ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...,"[ไฮยีน, , เอ็กซ์เพิร์ท, , แคร์, , น้ำยาปรับ...",ไฮยีน เอ็กซ์เพิร์ท แคร์ น้ำยาปรับผ้านุ่มสูตรเข...
3,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...,1-GDS-HYGIE-000000034,12,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...,"[ไฮยีน, , เอ็กซ์เพิร์ท, , แคร์, , ไลฟ์เนเจอ...",ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เนเจอร์ น้ำยาปรับผ...
4,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เซ้นท์ น้ำยาปรับผ้...,1-GDS-HYGIE-000000058,12,ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เซ้นท์ น้ำยาปรับผ้...,"[ไฮยีน, , เอ็กซ์เพิร์ท, , แคร์, , ไลฟ์เซ้นท...",ไฮยีน เอ็กซ์เพิร์ท แคร์ ไลฟ์เซ้นท์ น้ำยาปรับผ้...


In [42]:
df_vendor = df_vendor.merge(df_bzb[['OtherDescription', 'SKUCode']], how='left', left_on='return_SKUCode', right_on='SKUCode')
df_vendor.head()

,ProductCode,ProductName,Qty,Pack,Weight,Vendor_Name_Clean01,Vendor_Name_Clean02,Vendor_Name_Clean03,BZB_Qty,matching_indices,max_cosine_similarity,return_SKUCode,OtherDescription,SKUCode
0,1101155228,วิกซอล โกลด์ 900 (2+1),12,1,12.5,วิกซอล โกลด์ 900 (2+1),"[วิกซอล, , โกลด์, , 900, , (2+1)]",วิกซอล โกลด์ 900 (2+1),4,"[27, 28, 29, 30, 31, 32, 37, 48]",0.482393,1-GDS-VIXOL-000000014,วิกซอล โกลด์ นำยาล้างห้องน้ำและสุขภัณฑ์ 900 มล...,1-GDS-VIXOL-000000014
1,1102043121,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,12,1,9.8,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,"[วิกซอล, , ออกซี่, , มิ้นท์, , 700, , มล, ...",วิกซอล ออกซี่ มิ้นท์ 700 มล . 3S,4,"[27, 28, 29, 30, 31, 32, 37, 48]",0.296545,1-GDS-VIXOL-000000025,วิกซอล ออกซี่ ผลิตภัณฑ์ล้างห้องน้ำและสุขภัณฑ์ ...,1-GDS-VIXOL-000000025
2,1201033318,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,12,1,8.5,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,"[วิช, -, ถูพื้น, , ทรีดี, , ชมพู, , 650, ,...",วิช ถูพื้น ทรีดี ชมพู 650 ซอง,12,"[0, 1, 2, 3, 4, 5, 7, 8, 9, 16, 17, 19, 20, 21...",0.198344,1-GDS-WHIZ0-000000004,วิซ 3D ผลิตภัณฑ์ทำความสะอาดพื้นสูตรเข้มข้น กลิ...,1-GDS-WHIZ0-000000004
3,1201053312,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,12,1,8.5,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,"[วิช, -, ถูพื้น, , ทรีดี, , ฟ้า, , 650, , ...",วิช ถูพื้น ทรีดี ฟ้า 650 ซอง,12,"[0, 1, 2, 3, 4, 5, 7, 8, 9, 16, 17, 19, 20, 21...",0.135273,1-GDS-WHIZ0-000000005,วิซ 3D ผลิตภัณฑ์ทำความสะอาดพื้นสูตรเข้มข้น กลิ...,1-GDS-WHIZ0-000000005
4,1201133155,วิช-ถูพื้น เนเจอร์ ชมพู 5200,4,1,21.9,วิช-ถูพื้น เนเจอร์ ชมพู 5200,"[วิช, -, ถูพื้น, , เนเจอร์, , ชมพู, , 5200]",วิช ถูพื้น เนเจอร์ ชมพู 5200,4,"[27, 28, 29, 30, 31, 32, 37, 48]",0.139477,1-GDS-HYGIE-000000110,ผลิตภัณฑ์ปรับผ้านุ่มสูตรมาตรฐาน ไฮยีน กลิ่น พิ...,1-GDS-HYGIE-000000110


In [43]:
resulted_df = df_vendor.copy()
resulted_df = resulted_df[['ProductName', 'OtherDescription', 'max_cosine_similarity']]
resulted_df.head()

,ProductName,OtherDescription,max_cosine_similarity
0,วิกซอล โกลด์ 900 (2+1),วิกซอล โกลด์ นำยาล้างห้องน้ำและสุขภัณฑ์ 900 มล...,0.482393
1,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,วิกซอล ออกซี่ ผลิตภัณฑ์ล้างห้องน้ำและสุขภัณฑ์ ...,0.296545
2,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,วิซ 3D ผลิตภัณฑ์ทำความสะอาดพื้นสูตรเข้มข้น กลิ...,0.198344
3,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,วิซ 3D ผลิตภัณฑ์ทำความสะอาดพื้นสูตรเข้มข้น กลิ...,0.135273
4,วิช-ถูพื้น เนเจอร์ ชมพู 5200,ผลิตภัณฑ์ปรับผ้านุ่มสูตรมาตรฐาน ไฮยีน กลิ่น พิ...,0.139477


In [44]:
resulted_df

,ProductName,OtherDescription,max_cosine_similarity
0,วิกซอล โกลด์ 900 (2+1),วิกซอล โกลด์ นำยาล้างห้องน้ำและสุขภัณฑ์ 900 มล...,0.482393
1,วิกซอล ออกซี่ มิ้นท์ 700 มล. 3S,วิกซอล ออกซี่ ผลิตภัณฑ์ล้างห้องน้ำและสุขภัณฑ์ ...,0.296545
2,วิช-ถูพื้น ทรีดี ชมพู 650 ซอง,วิซ 3D ผลิตภัณฑ์ทำความสะอาดพื้นสูตรเข้มข้น กลิ...,0.198344
3,วิช-ถูพื้น ทรีดี ฟ้า 650 ซอง,วิซ 3D ผลิตภัณฑ์ทำความสะอาดพื้นสูตรเข้มข้น กลิ...,0.135273
4,วิช-ถูพื้น เนเจอร์ ชมพู 5200,ผลิตภัณฑ์ปรับผ้านุ่มสูตรมาตรฐาน ไฮยีน กลิ่น พิ...,0.139477
5,ไฮยีน-บลีช ชมพู 1500,ไฮยีน ผลิตภัณฑ์ขจัดคราบสำหรับผ้าขาว สีชมพู 150...,0.377137
6,ไฮยีน-บลีชผ้าสี 1000,ไฮยีน ผลิตภัณฑ์ขจัดคราบสำหรับผ้าขาวและผ้าสี กล...,0.319591
7,ไฮยีน ปากกาขจัดคราบ ฟ้า 10,ไฮยีน ปากกาขจัดคราบ 10 มล.,0.822039
8,ไฮยีน ซักผ้า ฟอร์เอเวอร์ บลูม 1400,ไฮยีน เอ็กซ์เพิร์ท วอช ผลิตภัณฑ์ซักผ้าชนิดน้ำ ...,0.423375
9,ไฮยีนซักผ้า เลิฟลี่ 2800,ไฮยีน เอ็กซ์เพิร์ท วอช ผลิตภัณฑ์ซักผ้าชนิดน้ำ ...,0.302345


In [45]:
resulted_df.to_excel("output/resulted_data.xlsx", index=False)